In [1]:
import json
import requests

In [2]:
def extract():
    f = open('tmdb.json')
    if f:
        return json.loads(f.read());

In [4]:
movieDict = extract()
analysis = {
    "analyser": {
        "default": {
            "type": "english"
        }
    }
}

In [5]:
def reindex(analysisSettings={}, mappingSettings={}, movieDict={}):
    settings = {
        "number_of_ shards": 1,
        "index": {
            "analysis": analysisSettings,
        }
    }
    if mappingSettings:
        settings['mappings'] = mappingSettings
        
    resp = requests.delete('http://192.168.32.10:9200/tmdb')
    resp = requests.put('http://192.168.32.10:9200/tmdb', data=json.dumps(settings))
    bulkMovies = ""
    for id, movie in movieDict.iteritems():
        addCmd = {"index": {"_index": "tmdb",
                           "_type": "movie",
                           "_id": movie["id"]}}
        bulkMovies += json.dumps(addCmd) + "\n" + json.dumps(movie) + "\n"
    resp = requests.post("http://192.168.32.10:9200/_bulk", data=bulkMovies)

In [6]:
reindex(analysisSettings=analysis, mappingSettings=None, movieDict=movieDict)

In [21]:
usersSearch = 'ship ice'
query = {
    "size": "20",
    "query": {
        "multi_match": {
            "query": usersSearch,
            "fields": ["title^10", "overview"],
        }
    }
}

In [22]:
def search(query):
    url = 'http://192.168.32.10:9200/tmdb/movie/_search'
    httpResp = requests.get(url, data=json.dumps(query))
    searchHits = json.loads(httpResp.text)['hits']
    print "Num\tRelevance Score \t\tMovie Title"
    for idx, hit in enumerate(searchHits['hits']):
        print "%s\t%s\t\t%s" % (idx + 1, hit['_score'], hit['_source']['title'])

In [23]:
search(query)

Num	Relevance Score 		Movie Title
1	1.5027361		Ghost Ship
2	1.381936		Ice Age
3	1.2021888		Ice Age: The Meltdown
4	1.1275065		Ice Age: Continental Drift
5	0.99401945		Ice Age: A Mammoth Christmas
6	0.84628254		Ice Age: Dawn of the Dinosaurs
7	0.069090545		Waterworld
8	0.032050554		Blades of Glory
9	0.029610232		K-19: The Widowmaker
10	0.029610232		The Colony
11	0.028187662		Batman & Robin
12	0.027206324		The Legend of 1900
13	0.026577717		xXx: State of the Union
14	0.02538728		Serenity
15	0.024675194		Captain Phillips
16	0.024649087		King Kong
17	0.02440135		WALL·E
18	0.02287776		G.I. Joe: The Rise of Cobra
19	0.022663165		For Your Eyes Only
20	0.022439398		The Chronicles of Narnia: The Voyage of the Dawn Treader


#### Snippet of Space Jam document

In [25]:
spaceJamId = 2300
httpResp = requests.get('http://192.168.32.10:9200/tmdb/movie/%s' % spaceJamId)
spaceJamDocument = json.loads(httpResp.text)

print json.dumps(spaceJamDocument['_source'], indent=True)

{
 "poster_path": "/9T9ucCk6wO0crRBUIkBJMRAVcKp.jpg", 
 "production_countries": [
  {
   "iso_3166_1": "US", 
   "name": "United States of America"
  }
 ], 
 "revenue": 230000000, 
 "overview": "Michael Jordan agrees to help the Looney Tunes play a basketball game against alien slavers to determine their freedom.", 
 "video": false, 
 "id": 2300, 
 "genres": [
  {
   "id": 16, 
   "name": "Animation"
  }, 
  {
   "id": 35, 
   "name": "Comedy"
  }, 
  {
   "id": 18, 
   "name": "Drama"
  }, 
  {
   "id": 14, 
   "name": "Fantasy"
  }, 
  {
   "id": 10751, 
   "name": "Family"
  }
 ], 
 "title": "Space Jam", 
 "tagline": "Get ready to jam.", 
 "vote_count": 275, 
 "homepage": "", 
 "belongs_to_collection": null, 
 "original_language": "en", 
 "status": "Released", 
 "spoken_languages": [
  {
   "iso_639_1": "cs", 
   "name": "\u010cesk\u00fd"
  }, 
  {
   "iso_639_1": "fr", 
   "name": "Fran\u00e7ais"
  }, 
  {
   "iso_639_1": "pl", 
   "name": "Polski"
  }, 
  {
   "iso_639_1": "en", 


### Star Trek query

In [28]:
usersSearch = 'patrick stewart'
query = {
    "query": {
        "multi_match": {
            "query": usersSearch,
            "fields": ["title", "overview", "cast.name", "directors.name^0.1"],
            "type": "best_fields"
        }
    }
}
search(query)

Num	Relevance Score 		Movie Title
1	0.57560366		Star Trek: Insurrection
2	0.57560366		Excalibur
3	0.56755596		Vertigo
4	0.53159344		Gnomeo & Juliet
5	0.52390707		Star Trek: Nemesis
6	0.5086731		Star Trek: First Contact
7	0.47966972		The Wolverine
8	0.47966972		Dune
9	0.4729633		Star Trek: Generations
10	0.4729633		Panic Room


### Analysis extracting English bigrams